# WaygateDataReader demo

The WaygateDataReader reads Waygate .pca data files and loads:
- Geometry data into the CIL geomeotry format
- Projection data which has already been normalised with flat and dark fields cast to float 32

The dataset is a CT scan of a plug collected by Evelien Zwanenburg and Jay Warnett at WMG on a Waygate Phoenix V|tome|x M300. The dataset can be downloaded from https://zenodo.org/records/14993754

This notebook was developed with CIL v25.0.0

In [ ]:
import cil
cil.version.version

First import some relevant python libraries

In [ ]:
import os
from readers.WaygateDataReader import WaygateDataReader

from cil.processors import CentreOfRotationCorrector, TransmissionAbsorptionConverter, Padder
from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer
from cil.recon import FDK

Load the Waygate dataset

In [ ]:
path = '/mnt/share/stfc.ac.uk/tomography/hackathon-11-25/waygate_plug/acq_data_750'
filename = os.path.join(path, 'Plug 750.pca')
data = WaygateDataReader(file_name = filename).read()

Visualise and print the geometry

In [ ]:
show_geometry(data.geometry)

In [ ]:
print(data.geometry)

Scroll through the data using `islicer`

In [ ]:
islicer(data)

The data is already normalised between 0-1, next we convert to absorption

In [ ]:
data_norm= TransmissionAbsorptionConverter()(data)
show2D([data, data_norm], ['Transmission data', 'Absorption data'])

Now we use FDK from the CIL recon class to reconstruct the data

In [ ]:
fdk = FDK(data_norm)                                           
recon = fdk.run()

Visualise the central slice of the reconstruction

In [ ]:
show2D(recon)

We notice a lot of artefacts in the reconstruction likely due to the metal parts of the plug. We can optimise the centre of rotation using the `CentreOfRotationCorrector.image_sharpness` method

In [ ]:
print(f"Centre of rotation before = {data.geometry.get_centre_of_rotation(distance_units='pixels')}")
data_centred = CentreOfRotationCorrector.image_sharpness(backend='tigre')(data_norm)
print(f"Centre of rotation after = {data.geometry.get_centre_of_rotation(distance_units='pixels')}")

Re-run the reconstruction

In [ ]:
fdk = FDK(data_centred)                                             
recon_centred = fdk.run()

Zoom in on a small area to see if the image sharpness has improved

In [ ]:
show2D([recon.array[300,200:400,320:520], recon_centred.array[300,200:400,320:520]])

Use `islicer` to scroll through the reconstruction, we can use the range sliders to zoom in on specific details 

In [ ]:
islicer(recon_centred)

Now we test if we can load in only a subset of the data using the `roi` argument

First we select a smaller region around the plug by restricting the vertical range

In [ ]:
data_roi = WaygateDataReader(filename, roi={'vertical':(90, 650)}).read()
data_roi = TransmissionAbsorptionConverter()(data_roi)
print(f'Original shape = {data.shape}')
print(f'ROI shape = {data_roi.shape}')
show2D([data_norm, data_roi])

Next we try binning the data using `roi={'horizontal':(0,-1,10), 'vertical':(0,-1,10)}` to specify the full range in vertical and horizontal but binned 10x.

In [ ]:
data_binned = WaygateDataReader(filename, roi={'horizontal':(0,-1,10), 'vertical':(0,-1,10)}).read()
data_binned = TransmissionAbsorptionConverter()(data_binned)
show2D([data_norm, data_binned])

Try reducing the number of angles loaded using `roi={'angle' : (0, -1, 3)}` and specifying `mode='slice'` to read only every 3rd projection. Here we visualise the ROI by looking at the sinogram.

In [ ]:
data_sliced = WaygateDataReader(filename, roi={'angle' : (0, -1, 3)}, mode='slice').read()
data_sliced = TransmissionAbsorptionConverter()(data_sliced)
print(f'Original shape = {data.shape}')
print(f'ROI shape = {data_sliced.shape}')
show2D([data_norm, data_sliced], slice_list = ('vertical', 350))

Try reconstructing the sliced data

In [ ]:
fdk = FDK(data_sliced)                                             
recon_sliced = fdk.run(verbose=False)
show2D([recon, recon_sliced])

Next we use `fliplr=True` to test changing the origin of the dataset

In [ ]:
data_flip = WaygateDataReader(filename, roi={'vertical':(50, 700)}, fliplr=True).read()
data_flip= TransmissionAbsorptionConverter()(data_flip)
show2D([data_norm, data_flip])

And also check there is no impact on the reconstruction

In [ ]:
fdk = FDK(data_flip)                                             
recon_flip = fdk.run(verbose=False)
show2D([recon, recon_flip])